![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important.

For this project, you're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

### Goal:
Analyze the `manufacturing_parts` table and determine whether the manufacturing process is performing within acceptable control limits:

- Create an alert that flags whether the height of a product is within the control limits for each operator using the formulas provided in the workbook.
- The final query should return the following fields: `operator`, `row_number`, `height`, `avg_height`, `stddev_height`, `ucl`, `lcl`, `alert`, and be ordered by the `item_no`.
- The `alert` column will be your boolean flag.
- Use a window function of length 5 to calculate the control limits, considering rows up to and including the current row; incomplete windows should be excluded from the final query output.
- Save this DataFrame as `alerts`.

> **Note**: Creating new cells in the workbook will rename the DataFrame. Make sure that your final solution uses the name provided: `alerts`.

In [49]:
import pandas as pd
import duckdb as db

In [51]:
parts = pd.read_csv("parts.csv")

In [53]:
parts.head()

,item_no,length,width,height,operator
0,1,102.67,49.53,19.69,Op-1
1,2,102.50,51.42,19.63,Op-1
2,3,95.37,52.25,21.51,Op-1
3,4,94.77,49.24,18.60,Op-1
4,5,104.26,47.90,19.46,Op-1


In [55]:
# Flag whether the height of a product is within the control limits
result = db.query("""
    SELECT
	b.*,
	CASE
		WHEN 
			b.height NOT BETWEEN b.lcl AND b.ucl
		THEN TRUE
		ELSE FALSE
	END as alert
FROM (
	SELECT
		a.*, 
		a.avg_height + 3*a.stddev_height/SQRT(5) AS ucl, 
		a.avg_height - 3*a.stddev_height/SQRT(5) AS lcl  
	FROM (
		SELECT 
			operator,
			ROW_NUMBER() OVER w AS row_number, 
			height, 
			AVG(height) OVER w AS avg_height, 
			STDDEV(height) OVER w AS stddev_height
		FROM parts 
		WINDOW w AS (
			PARTITION BY operator 
			ORDER BY item_no 
			ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
		)
	) AS a
	WHERE a.row_number >= 5
) AS b;
""").to_df()

result.head()

,operator,row_number,height,avg_height,stddev_height,ucl,lcl,alert
0,Op-19,5,20.28,20.358,1.209595,21.980842,18.735158,False
1,Op-19,6,21.02,20.162,0.922291,21.399383,18.924617,False
2,Op-19,7,19.49,20.324,0.617762,21.152815,19.495185,True
3,Op-19,8,20.92,20.506,0.635909,21.359161,19.652839,False
4,Op-19,9,19.25,20.192,0.806765,21.274389,19.109611,False


In [59]:
# Alternative: Flag whether the height of a product is within the control limits
result_alt = db.query("""
   WITH rolling_calculations AS (
    SELECT 
        operator,
        height,
        ROW_NUMBER() OVER (PARTITION BY operator ORDER BY item_no) AS row_number,
        AVG(height) OVER (
            PARTITION BY operator 
            ORDER BY item_no 
            ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
        ) AS avg_height,
        STDDEV(height) OVER (
            PARTITION BY operator 
            ORDER BY item_no 
            ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
        ) AS stddev_height
    FROM parts
)
SELECT 
    *,
    avg_height + 3 * stddev_height / SQRT(5) AS ucl,
    avg_height - 3 * stddev_height / SQRT(5) AS lcl,
    CASE 
        WHEN height NOT BETWEEN 
            (avg_height - 3 * stddev_height / SQRT(5)) AND 
            (avg_height + 3 * stddev_height / SQRT(5))
        THEN TRUE 
        ELSE FALSE 
    END AS alert
FROM rolling_calculations
WHERE row_number >= 5;
""").to_df()

result_alt.head()

,operator,height,row_number,avg_height,stddev_height,ucl,lcl,alert
0,Op-19,20.28,5,20.358,1.209595,21.980842,18.735158,False
1,Op-19,21.02,6,20.162,0.922291,21.399383,18.924617,False
2,Op-19,19.49,7,20.324,0.617762,21.152815,19.495185,True
3,Op-19,20.92,8,20.506,0.635909,21.359161,19.652839,False
4,Op-19,19.25,9,20.192,0.806765,21.274389,19.109611,False
